In [3]:
import pandas as pd

In [4]:
try:
    df = pd.read_csv('/home/dean/git/SQL/layoffs_project/layoffs.csv')
except FileNotFoundError:
    print("Error: 'layoffs.csv' not found. Please make sure the file is in the correct directory.")
    exit()

Data Cleaning//

In [5]:
# 1. Remove Duplicates
print("1. Removing Duplicates:")
print(f"Number of rows before removing duplicates: {len(df)}")
df_no_duplicates = df.drop_duplicates()
print(f"Number of rows after removing duplicates: {len(df_no_duplicates)}")

1. Removing Duplicates:
Number of rows before removing duplicates: 2361
Number of rows after removing duplicates: 2356


In [6]:
# 2. Standardize the data

# Standardize company names (remove leading/trailing spaces)
df_no_duplicates.loc[:, 'company'] = df_no_duplicates['company'].str.strip()
print("Standardized company names (removed unnecessary spaces).")

Standardized company names (removed unnecessary spaces).


In [7]:
# Standardize industry (handle 'Crypto currency' and variations)
df_no_duplicates.loc[:,'industry'] = df_no_duplicates['industry'].str.replace(r'.*crypto.*', 'Crypto', regex=True)
print("Standardized industry names (grouped crypto variations).")

Standardized industry names (grouped crypto variations).


In [8]:
# Standardize country (remove trailing '.')
df_no_duplicates.loc[:,'country'] = df_no_duplicates['country'].str.rstrip('.')
print("Standardized country names (removed trailing periods from 'USA.').")

Standardized country names (removed trailing periods from 'USA.').


In [9]:
# Convert 'date' column to datetime objects
df_no_duplicates.loc[:,'date'] = pd.to_datetime(df_no_duplicates['date'], format='%m/%d/%Y')
print("Converted 'date' column to datetime objects.")

Converted 'date' column to datetime objects.


In [10]:
# 3. Deal with Null values, blanks, and removal of unnecessary artifacts

# Replace blank strings in 'industry' with NaN
df_no_duplicates.loc[:,'industry'] = df_no_duplicates['industry'].replace('', pd.NA)

In [11]:
# Fill missing 'industry' values based on company name

def fill_missing_industry(series):
    if pd.isna(series.iloc[0]):
        non_na_values = series.dropna().unique()
        if len(non_na_values) == 1:
            return non_na_values[0]
    return series.iloc[0]

df_no_duplicates.loc[:,'industry'] = df_no_duplicates.groupby('company')['industry'].transform(fill_missing_industry)
print("Filled missing 'industry' values based on company name where possible.")

Filled missing 'industry' values based on company name where possible.


In [12]:
# Remove rows where both 'total_laid_off' and 'percentage_laid_off' are NaN
df_cleaned = df_no_duplicates.dropna(subset=['total_laid_off', 'percentage_laid_off'], how='all')
print(f"Number of rows after handling critical nulls: {len(df_cleaned)}")
print("\nCleaned Data (first few rows):")
print(df_cleaned.head())


Number of rows after handling critical nulls: 1995

Cleaned Data (first few rows):
     company       location     industry  total_laid_off  percentage_laid_off  \
0  Atlassian         Sydney        Other           500.0                 0.05   
1   SiriusXM  New York City        Media           475.0                 0.08   
2     Alerzo         Ibadan       Retail           400.0                  NaN   
3     UpGrad         Mumbai    Education           120.0                  NaN   
4       Loft      Sao Paulo  Real Estate           340.0                 0.15   

                  date     stage        country  funds_raised_millions  
0  2023-03-06 00:00:00  Post-IPO      Australia                  210.0  
1  2023-03-06 00:00:00  Post-IPO  United States                  525.0  
2  2023-03-06 00:00:00  Series B        Nigeria                   16.0  
3  2023-03-06 00:00:00   Unknown          India                  631.0  
4  2023-03-03 00:00:00   Unknown         Brazil                  

Exploratory Data Analysis//

In [13]:
# What was the largest lay off in a single occurrence?
largest_layoff = df_cleaned['total_laid_off'].max()
print(f"Largest layoff in a single occurrence: {largest_layoff}")

Largest layoff in a single occurrence: 12000.0


In [14]:
# Who had the most people laid off as a percentage of the company?
most_laid_off_percent = df_cleaned['percentage_laid_off'].max()
print("\nCompanies with the highest percentage laid off:")
print(df_cleaned[df_cleaned['percentage_laid_off'] == most_laid_off_percent]['company'].tolist())




Companies with the highest percentage laid off:
['Kandela', 'DUX Education', 'Locomation', 'Fipola', 'EMX Digital', 'WeTrade', 'Medly', 'Openpay', 'Mode Global', 'Earth Rides', 'Britishvolt', 'Arch Oncology', 'Lantern', 'Wyre', 'YourGrocer', 'Brodmann17', 'Digital Surge', 'Lora DiCarlo', 'Bitfront', 'BlockFi', 'Assure', 'GoodGood', 'GloriFi', 'Kite', 'Deliveroo Australia', 'Protocol', 'Nirvana Money', 'Wavely', 'Faze Medicines', 'Planetly', 'Fifth Season', 'GoNuts', 'Nuri', 'Flux Systems', 'Qin1', 'Playdots', 'Pastel', 'Pesto', 'Kitty Hawk', 'Propzy', 'CommonBond', 'Lido Learning', 'Simple Feast', 'The Wing', 'Reali', 'ShopX', 'Edmodo', 'Pollen', 'Haus', 'Perceptive Automata', 'Metigy', 'Yabonza', 'Soluto', 'Airlift', 'Butler Hospitality', 'WanderJaunt', 'Crejo.Fun', 'Gavelytics', 'Volt Bank', 'Kune', 'SuperLearn', 'SummerBio', 'JetClosing', 'The Grommet', 'Kaodim', 'Udayy', 'BeyondMinds', 'Subspace', 'SEND', 'Halcyon Health', 'Ahead', 'Fast', 'Ozy Media', 'Katerra', 'Madefire', 'Lime

In [15]:
# What company had the largest lay off in total?
total_layoffs_by_company = df_cleaned.groupby('company')['total_laid_off'].sum().sort_values(ascending=False)
print("\nCompany with the largest total layoffs:")
print(total_layoffs_by_company)



Company with the largest total layoffs:
company
Amazon            18150.0
Google            12000.0
Meta              11000.0
Salesforce        10090.0
Microsoft         10000.0
                   ...   
Reach                 0.0
DriveWealth           0.0
Locomation            0.0
Dude Solutions        0.0
PagSeguro             0.0
Name: total_laid_off, Length: 1633, dtype: float64


In [16]:
# What is the time frame of the data we got?
datetime_dates = df_cleaned['date'][pd.to_datetime(df_cleaned['date'], errors='coerce').notna()]
min_date = datetime_dates.min()
max_date = datetime_dates.max()
print(f"\nTime frame of the data: {min_date.strftime('%Y-%m-%d')} to {max_date.strftime('%Y-%m-%d')}")



Time frame of the data: 2020-03-11 to 2023-03-06


In [37]:
# Which industry had the most laying off?
total_layoffs_by_industry = df_cleaned.groupby('industry')['total_laid_off'].sum().sort_values(ascending=False)
print("\nIndustry with the most layoffs:")
print(total_layoffs_by_industry.head(1))


Industry with the most layoffs:
industry
Consumer    45541.0
Name: total_laid_off, dtype: float64


In [38]:
# Which country had the most laying off?
total_layoffs_by_country = df_cleaned.groupby('country')['total_laid_off'].sum().sort_values(ascending=False)
print("\nCountry with the most layoffs:")
print(total_layoffs_by_country.head(1))


Country with the most layoffs:
country
United States    256559.0
Name: total_laid_off, dtype: float64


In [ ]:
# What year had the most layoffs
total_layoffs_by_year = df_cleaned.dropna(subset=['date']).groupby(df_cleaned['date'].dt.year)['total_laid_off'].sum().sort_values(ascending=False)
print("\nYear with the most layoffs:")
print(total_layoffs_by_year.head(1))



Year with the most layoffs:
date
2022.0    160661.0
Name: total_laid_off, dtype: float64


In [45]:
# What stage had the most layoffs?
total_layoffs_by_stage = df_cleaned.groupby('stage')['total_laid_off'].sum().sort_values(ascending=False)
print("\nStage with the most layoffs:")
print(total_layoffs_by_stage.head(1))


Stage with the most layoffs:
stage
Post-IPO    204132.0
Name: total_laid_off, dtype: float64


In [48]:
# How many layoffs happened? show by months
df_cleaned['month_year'] = df_cleaned['date'].dt.strftime('%Y-%m')
total_layoffs_by_month = df_cleaned.groupby('month_year')['total_laid_off'].sum().sort_values(ascending=False)
print("\nTotal layoffs by month:")
print(total_layoffs_by_month)


Total layoffs by month:
month_year
2023-01    84714.0
2022-11    53451.0
2023-02    36493.0
2020-04    26710.0
2020-05    25804.0
2022-10    17406.0
2022-06    17394.0
2022-07    16223.0
2022-08    13055.0
2022-05    12885.0
2022-12    10329.0
2020-03     9628.0
2020-06     7627.0
2020-07     7112.0
2021-01     6813.0
2022-09     5881.0
2022-03     5714.0
2023-03     4470.0
2022-04     4128.0
2022-02     3685.0
2021-06     2434.0
2021-11     2070.0
2020-08     1969.0
2021-08     1867.0
2021-12     1200.0
2021-02      868.0
2020-12      852.0
2020-09      609.0
2022-01      510.0
2020-10      450.0
2021-04      261.0
2020-11      237.0
2021-09      161.0
2021-07       80.0
2021-03       47.0
2021-10       22.0
Name: total_laid_off, dtype: float64


/tmp/ipykernel_11363/1418196045.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['month_year'] = df_cleaned['date'].dt.strftime('%Y-%m')


In [49]:
# Show progress of layoffs with each month passing and accumulate the sum
rolling_layoffs = df_cleaned.groupby('month_year')['total_laid_off'].sum().cumsum()
print("\nRolling total of layoffs by month:")
print(rolling_layoffs)


Rolling total of layoffs by month:
month_year
2020-03      9628.0
2020-04     36338.0
2020-05     62142.0
2020-06     69769.0
2020-07     76881.0
2020-08     78850.0
2020-09     79459.0
2020-10     79909.0
2020-11     80146.0
2020-12     80998.0
2021-01     87811.0
2021-02     88679.0
2021-03     88726.0
2021-04     88987.0
2021-06     91421.0
2021-07     91501.0
2021-08     93368.0
2021-09     93529.0
2021-10     93551.0
2021-11     95621.0
2021-12     96821.0
2022-01     97331.0
2022-02    101016.0
2022-03    106730.0
2022-04    110858.0
2022-05    123743.0
2022-06    141137.0
2022-07    157360.0
2022-08    170415.0
2022-09    176296.0
2022-10    193702.0
2022-11    247153.0
2022-12    257482.0
2023-01    342196.0
2023-02    378689.0
2023-03    383159.0
Name: total_laid_off, dtype: float64


In [50]:
# Identify the top 5 companies with the highest number of layoffs for each year
layoffs_by_company_year = df_cleaned.groupby(['company', df_cleaned['date'].dt.year])['total_laid_off'].sum().reset_index()
layoffs_by_company_year = layoffs_by_company_year.rename(columns={'date': 'year'})
layoffs_by_company_year['rank'] = layoffs_by_company_year.groupby('year')['total_laid_off'].rank(method='dense', ascending=False)
top_5_companies_by_year = layoffs_by_company_year[layoffs_by_company_year['rank'] <= 5]
print("\nTop 5 companies with the highest number of layoffs for each year:")
print(top_5_companies_by_year.sort_values(by=['year', 'rank']))


Top 5 companies with the highest number of layoffs for each year:
          company    year  total_laid_off  rank
1631         Uber  2020.0          7525.0   1.0
194   Booking.com  2020.0          4375.0   2.0
653       Groupon  2020.0          2800.0   3.0
1503       Swiggy  2020.0          2250.0   4.0
38         Airbnb  2020.0          1900.0   5.0
247     Bytedance  2021.0          3600.0   1.0
809       Katerra  2021.0          2434.0   2.0
1799       Zillow  2021.0          2000.0   3.0
757     Instacart  2021.0          1877.0   4.0
1746  WhiteHat Jr  2021.0          1800.0   5.0
979          Meta  2022.0         11000.0   1.0
56         Amazon  2022.0         10150.0   2.0
315         Cisco  2022.0          4100.0   3.0
1167      Peloton  2022.0          4084.0   4.0
283       Carvana  2022.0          4000.0   5.0
1181      Philips  2022.0          4000.0   5.0
643        Google  2023.0         12000.0   1.0
983     Microsoft  2023.0         10000.0   2.0
511      Ericsson  20